In [37]:
import numpy as np
import pandas as pd
import re
import json
from sklearn.model_selection import StratifiedKFold
import keras.backend as K
from keras.layers import Input, Dense, Embedding, TimeDistributed, Bidirectional, LSTM, merge, concatenate, Dropout
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras import metrics
from gensim.models.poincare import PoincareModel
from wikipedia2vec import Wikipedia2Vec

In [2]:
# Fix ramdom seed.
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)

In [7]:
# load Production sentence data
train_df = pd.read_csv("../data/train_split_words.csv", dtype={'_id': str})
valid_df = pd.read_csv("../data/valid_split_words.csv", dtype={'_id': str})

print("Number of train rows:", len(train_df))
print("True:", len(train_df[train_df.label == True]), "\tFalse:", len(train_df[train_df.label == False]))

print("Number of validete rows:", len(valid_df))
print("True:", len(valid_df[valid_df.label == True]), "\tFalse:", len(valid_df[valid_df.label == False]))

Number of train rows: 7435
True: 508 	False: 6927
Number of validete rows: 1564
True: 88 	False: 1476


In [11]:
wiki2vec = Wikipedia2Vec.load('../model/jawiki_20180420_300d.pkl')
poincare_model = PoincareModel.load("../model/poincare.model")

In [36]:
WORD_EMBEDDING_DIM = 300
WORD_LSTM_UNIT = 1024
ENTRIES_EMBEDDING_DIM = 10
FC_DIM = 512
DROPOUT_RATE = 0.5

In [41]:
def sentence2vec(s: str):
    return [_w2v(w) for w in m.parse(s).strip().split()]

def _w2v(w):
    try:
        return np.array(wiki2vec.get_word_vector(w).tolist())
    except KeyError:
        return np.zeros(WORD_EMBEDDING_DIM)

def compound2vec_poincare(_id: str):
    '''
    Using Poincaré embedding.
    '''
    try:
        return poincare_model.kv[].tolist()
    except KeyError:
        return np.zeros(ENTRIES_EMBEDDING_DIM)

In [42]:
X_train_words = pad_sequences(
    train_df.sentence.apply(lambda x: sentence2vec(x)).tolist()
    , dtype='float32'
    , padding='post'
    , truncating='pre'
    , maxlen=50
)
X_train_ontology = np.array(train_df._id.apply(lambda x: compound2vec(x)).tolist())
y_train = train_df.label.values


In [43]:
# check dimensions
print(x_train_words.shape)
print(x_train_entries.shape)
print(y_train.shape)

(7404, 50, 100)
(7404, 100)
(7404,)


In [14]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [44]:
word_embeddings = Input(shape=(None, WORD_EMBEDDING_DIM,), dtype='float32')
ontology_embeddings = Input(shape=(ONTOLOGY_EMBEDDING_DIM,), dtype='float32')

l_drop_word = Dropout(DROPOUT_RATE)(word_embeddings)
l_lstm = Bidirectional(LSTM(WORD_LSTM_UNIT))(l_drop_word)
x = concatenate([l_lstm, ontology_embeddings])

x = Dropout(DROPOUT_RATE)(x)
x = Dense(FC_DIM, activation='relu')(x)
x = Dropout(DROPOUT_RATE)(x)
x = Dense(FC_DIM, activation='relu')(x)
x = Dropout(DROPOUT_RATE)(x)

pred = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[premise_input, hypothesis_input], outputs=pred)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[metrics.binary_accuracy, f1])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 100)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               160800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total para

In [48]:
from keras.utils import plot_model
import pydot
plot_model(model, show_shapes=True, to_file='model3.png')

In [45]:
model.fit(x=[x_train_words, x_train_entries], y=y_train, epochs=40, batch_size=128)

Epoch 1/40
7404/7404 [==============================] - 11s 2ms/step - loss: 0.2748 - binary_accuracy: 0.9302 - f1: 0.0015
Epoch 2/40
7404/7404 [==============================] - 12s 2ms/step - loss: 0.2180 - binary_accuracy: 0.9341 - f1: 0.0000e+00
Epoch 3/40
7404/7404 [==============================] - 10s 1ms/step - loss: 0.1943 - binary_accuracy: 0.9341 - f1: 0.0000e+00
Epoch 4/40
7404/7404 [==============================] - 10s 1ms/step - loss: 0.1713 - binary_accuracy: 0.9340 - f1: 0.0029
Epoch 5/40
7404/7404 [==============================] - 10s 1ms/step - loss: 0.1739 - binary_accuracy: 0.9338 - f1: 0.0658
Epoch 6/40
7404/7404 [==============================] - 10s 1ms/step - loss: 0.1647 - binary_accuracy: 0.9357 - f1: 0.1886
Epoch 7/40
7404/7404 [==============================] - 10s 1ms/step - loss: 0.1617 - binary_accuracy: 0.9361 - f1: 0.2116
Epoch 8/40
7404/7404 [==============================] - 11s 1ms/step - loss: 0.1569 - binary_accuracy: 0.9358 - f1: 0.2416
Epoch 9/

In [47]:
predict = model.predict([x_valid_words, x_valid_entries])

In [3]:
def evaluation(pred_true, pred_false):
    TP = pred_true[pred_true.label == True].count()[0]
    FP = pred_true[pred_true.label == False].count()[0]
    TN = pred_false[pred_false.label == False].count()[0]
    FN = pred_false[pred_false.label == True].count()[0]

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F1 = 2 * precision * recall / (precision + recall)

    print("TP:", TP, "\tFP:", FP, "\tTN:", TN, "\tFN:", FN)
    print("Precision:", precision, "\tRecall:", recall, "\tF1:", F1)

In [49]:
pred_true = valid_df.loc[np.where(predict >= 0.5)[0]]
pred_false = valid_df.loc[np.where(predict < 0.5)[0]]
pred_true_uniq = pred_true.drop_duplicates(['_id', 'sentence'])
pred_false_uniq = pred_false.drop_duplicates(['_id', 'sentence'])

evaluation(pred_true, pred_false)

In [56]:
pred_true.values

array([['652294', '硫化窒素', '液体二酸化硫黄には分解せずに溶解する。', False],
       ['652294', '硫化窒素',
        '1837年にスーベイラン (Soubeiran) が二塩化硫黄のベンゼン溶液にアンモニアを通じることで初めて合成した。',
        True],
       ['652294', '硫化窒素',
        '4 SCl2 ＋ 16 NH3 ＋ 2 Cl2 → S4N4 ＋ 12 NH4Cl 二塩化二硫黄の二硫化炭素溶液にアンモニアを作用させてもできる。',
        True],
       ['652294', '硫化窒素', '塩酸または二塩化二硫黄と反応してチオ塩化チアジルを生じる。', False],
       ['652294', '硫化窒素', '四硫化四窒素を真空中で 300 ℃ に加熱すると生じる。', True],
       ['652294', '硫化窒素',
        '四硫化四窒素を硫黄とともに二硫化炭素中で混合させてオートクレーブ中 110 ℃ に加熱すると生じる。', True],
       ['497499', 'N-メチルピロリドン',
        'N-メチル-2-ピロリドンは、γ-ブチロラクトンとメチルアミンとを縮合させて得る 高い溶解性を持つため、特に高分子化学の分野を中心に様々な物質に対する溶媒として用いられる。',
        True],
       ['487740', '三塩化リン',
        '工業的には、塩素と白リンの三塩化リン溶液を加熱還流しながら、生成する三塩化リンを集める方法で合成される。', True],
       ['487740', '三塩化リン', '三塩化リンを酸化するとリン酸トリクロリドが得られる。', False],
       ['487740', '三塩化リン',
        'ウィッティヒ反応とホーナー・ワズワース・エモンズ反応は共にアルケンの合成法として重要である。', False],
       ['86438', 'プロピオン酸', '1-プロパノール、プロピオンアルデヒドの酸化によって得られる。', True],
       [

In [55]:
pred_false[pred_false.label == True].values

array([['652294', '硫化窒素',
        '6 SCl2 ＋ 16 NH3 → N4S4 ＋ 2 S ＋ 12 NH4Cl 二塩化硫黄の四塩化炭素溶液に塩素を加え、アンモニアを通しても生ずる。',
        True],
       ['652294', '硫化窒素', '二硫化二窒素を真空中で長時間放置すると生じる。', True],
       ['487740', '三塩化リン', '実験室ではより毒性の低い赤リンを使う。', True],
       ['86438', 'プロピオン酸',
        '語源は「最初の脂肪酸」という意味で、油脂の加水分解により得られる脂肪酸のうち、最も炭素数の少ないものであったことによる。',
        True],
       ['86438', 'プロピオン酸',
        '哺乳類の大腸やルーメンでは細菌が食物の中のセルロースやヘミセルロースを嫌気発酵し、プロピオン酸などの短鎖脂肪酸を生成しており、これが草食性動物の体内では重要なエネルギー源となっている。',
        True],
       ['86438', 'プロピオン酸', 'ウシなどの反芻動物は、第1胃で行われる糖質の発酵によって大量のプロピオン酸を生産する。',
        True],
       ['86438', 'プロピオン酸',
        '炭素数が奇数の脂肪酸はβ酸化により反応が進み、2個ずつの炭素がアセチルCoAとして生成し、最後に炭素数3個のプロピオニルCoAを生じる。',
        True],
       ['86438', 'プロピオン酸',
        'プロピオニルCoAは、プロピオン酸とCoAが結び付いたもので加水分解するとプロピオン酸が生じる。', True],
       ['1268469', 'ホスホエノールピルビン酸',
        'PEPは、2-ホスホグリセリン酸にエノラーゼが作用することにより生成する。', True],
       ['1268469', 'ホスホエノールピルビン酸',
        '2-ホスホグリセリン酸 ⟵ → ホスホエノールピルビン酸 → ピルビン酸 PEPは、オキサロ酢酸の脱炭酸によって